# 特定の日時のリストを取得する 

In [1]:
import jpholiday
import workdays
from pytz import timezone, utc
import datetime
import pickle

In [2]:
import numpy as np
import pandas as pd

In [3]:
def check_jst_datetimes_to_naive(*arg_datetimes):
    """
    ＊*今のところ，ローカルが東京でないnaiveなdatetimeはそのまま通してしまう
    引数のタイムゾーンが同じかどうかチェックし，存在するなら日本であるかチェック
    awareな場合は，naiveに変更
    """
    jst_timezone = timezone("Asia/Tokyo")
    tz_info_set = set([one_datetime.tzinfo for one_datetime in arg_datetimes])
    if len(tz_info_set) > 1:
        raise Exception("timezones are different")
        
    datetimes_tzinfo = list(tz_info_set)[0]
    
    if datetimes_tzinfo is not None:  # 長さが1のはず
        if timezone(str(datetimes_tzinfo)) != jst_timezone:
            raise Exception("timezones must be Asia/Tokyo")
        # naiveなdatetimeに変更
        arg_datetimes = [one_datetime.replace(tzinfo=None) for one_datetime in arg_datetimes]
        
    return tuple(arg_datetimes)

## 営業日・休日を取得する 

### 祝日のndarrayを取得する

以下が素直な実装だが，遅い．そこで一度しか呼ばないようにすべき

In [4]:
def get_holiday_jp(start_date, end_date, with_name=False):
    """
    期間を指定して祝日を取得
    
    start_date: datetime.date
        開始時刻のdate
    end_datetime: datetime.date
        終了時刻のdate
    eith_name: bool
        休日の名前を出力するかどうか
    to_date: bool
        出力をdatetime.datetimeにするかdatetime.dateにするか
    """
    assert isinstance(start_date, datetime.date) and isinstance(end_date, datetime.date)
    
    holydays_array = np.array(jpholiday.between(start_date, end_date))
    
    if not with_name:
        return holydays_array[:,0].copy()
    
    return holydays_array

In [5]:
start_date = datetime.date(2019, 1, 1)
end_date = datetime.date(2020, 12, 31)

holydays = get_holiday_jp(start_date, end_date, with_name=False)

In [6]:
holydays

array([datetime.date(2019, 1, 1), datetime.date(2019, 1, 14),
       datetime.date(2019, 2, 11), datetime.date(2019, 3, 21),
       datetime.date(2019, 4, 29), datetime.date(2019, 4, 30),
       datetime.date(2019, 5, 1), datetime.date(2019, 5, 2),
       datetime.date(2019, 5, 3), datetime.date(2019, 5, 4),
       datetime.date(2019, 5, 5), datetime.date(2019, 5, 6),
       datetime.date(2019, 7, 15), datetime.date(2019, 8, 11),
       datetime.date(2019, 8, 12), datetime.date(2019, 9, 16),
       datetime.date(2019, 9, 23), datetime.date(2019, 10, 14),
       datetime.date(2019, 10, 22), datetime.date(2019, 11, 3),
       datetime.date(2019, 11, 4), datetime.date(2019, 11, 23),
       datetime.date(2020, 1, 1), datetime.date(2020, 1, 13),
       datetime.date(2020, 2, 11), datetime.date(2020, 2, 23),
       datetime.date(2020, 2, 24), datetime.date(2020, 3, 20),
       datetime.date(2020, 4, 29), datetime.date(2020, 5, 3),
       datetime.date(2020, 5, 4), datetime.date(2020, 5, 5),


## 営業日を取得

内部で外部の関数等を利用するので，jitコンパイルはできない．そこで，あらかじめ数年分のholidayを得て置き，それと照らし合わせることにする．

In [7]:
class Option():
    """
    オプションの指定のためのクラス
    """
    def __init__(self):
        self._holiday_start_year = datetime.datetime.now().year-5
        self._holiday_end_year = datetime.datetime.now().year
        
        # 利用する休日のarray
        self._holidays_date_array =  get_holiday_jp(start_date=datetime.date(self._holiday_start_year,1,1),
                                                    end_date=datetime.date(self._holiday_end_year,12,31),
                                                    with_name=False,
                                                   )
        # 利用する休日のDatetimeIndex
        self._holidays_datetimeindex = pd.DatetimeIndex(self._holidays_date_array)
    
    @property
    def holiday_start_year(self):
        return self._holiday_start_year
    
    @property
    def holiday_end_year(self):
        return self._holiday_end_year
        
    @holiday_start_year.setter
    def holiday_start_year(self, year):
        self._holiday_start_year = year
        # 利用する休日のarray
        self._holidays_date_array =  get_holiday_jp(start_date=datetime.date(self._holiday_start_year,1,1),
                                                    end_date=datetime.date(self._holiday_end_year,12,31),
                                                    with_name=False,
                                                   )
        # 利用する休日のDatetimeIndex
        self._holidays_datetimeindex = pd.DatetimeIndex(self._holidays_date_array)
        
    @holiday_end_year.setter
    def holiday_end_year(self, year):
        self._holiday_end_year = year
        # 利用する休日のarray
        self._holidays_date_array =  get_holiday_jp(start_date=datetime.date(self._holiday_start_year,1,1),
                                                    end_date=datetime.date(self._holiday_end_year,12,31),
                                                    with_name=False,
                                                   )
        # 利用する休日のDatetimeIndex
        self._holidays_datetimeindex = pd.DatetimeIndex(self._holidays_date_array)
        
    @property
    def holidays_date_array(self):
        return self._holidays_date_array
    
    @property
    def holidays_datetimeindex(self):
        return self._holidays_datetimeindex

### Optionのコンストラクト 

In [8]:
option = Option()

### 指定した期間の営業日のndarrayを取得する

In [9]:
def get_workdays_jp(start_date, end_date, return_as="date", end_include=False):
    """
    営業日を取得
    
    start_date: datetime.date
        開始時刻のdate
    end_datetime: datetime.date
        終了時刻のdate
    return_as: str, defalt: 'dt'
        返り値の形式
        - 'dt':pd.DatetimeIndex
        - 'date': datetime.date array
    end_include: bool
        最終日も含めて出力するか
    """
    assert isinstance(start_date, datetime.date) and isinstance(end_date, datetime.date)
    # 返り値の形式の指定
    return_as_set = {"dt", "date"}
    if not return_as in return_as_set:
        raise Exception("return_as must be any in {}".format(return_as_set))
    
    # datetime.dateをpd.Timestampに変換(datetime.dateは通常pd.DatetimeIndexと比較できないため)
    start_timestamp = pd.Timestamp(start_date)
    end_timestamp = pd.Timestamp(end_date)
    
    # 期間中のholidayを取得
    holidays_in_span_index = (start_timestamp<=option.holidays_datetimeindex)&(option.holidays_datetimeindex<end_timestamp)  # DatetimeIndexを使うことに注意
    holidays_in_span_array = option.holidays_date_array[holidays_in_span_index]  # ndarrayを使う

    # 期間中のdatetimeのarrayを取得
    if end_include:
        days_datetimeindex = pd.date_range(start=start_date, end=end_date, freq="D")  # 最終日も含める
    else:
        days_datetimeindex = pd.date_range(start=start_date, end=end_date-datetime.timedelta(days=1), freq="D")  # 最終日は含めない
    
    
    # 休日に含まれないもの，さらに土日に含まれないもののboolインデックスを取得
    holiday_bool_array = np.in1d(days_datetimeindex.date, holidays_in_span_array)  # 休日であるかのブール(pd.DatetimeIndex.isin)でもいい
    sun_or_satur_bool_array = (days_datetimeindex.weekday==5) | (days_datetimeindex.weekday==6)  # 土曜日or日曜日
    
    workdays_bool_array = (~holiday_bool_array)&(~sun_or_satur_bool_array)  # 休日でなく土日でない
    
    workdays_datetimeindex = days_datetimeindex[workdays_bool_array].copy()
    if return_as=="dt":
        return workdays_datetimeindex
    elif return_as=="date":
        return workdays_datetimeindex.date

In [10]:
start_date = datetime.datetime(2019, 1, 1)
end_date = datetime.datetime(2019, 12, 31)

workdays = get_workdays_jp(start_date, end_date, return_as="date", end_include=False)

In [11]:
workdays

array([datetime.date(2019, 1, 2), datetime.date(2019, 1, 3),
       datetime.date(2019, 1, 4), datetime.date(2019, 1, 7),
       datetime.date(2019, 1, 8), datetime.date(2019, 1, 9),
       datetime.date(2019, 1, 10), datetime.date(2019, 1, 11),
       datetime.date(2019, 1, 15), datetime.date(2019, 1, 16),
       datetime.date(2019, 1, 17), datetime.date(2019, 1, 18),
       datetime.date(2019, 1, 21), datetime.date(2019, 1, 22),
       datetime.date(2019, 1, 23), datetime.date(2019, 1, 24),
       datetime.date(2019, 1, 25), datetime.date(2019, 1, 28),
       datetime.date(2019, 1, 29), datetime.date(2019, 1, 30),
       datetime.date(2019, 1, 31), datetime.date(2019, 2, 1),
       datetime.date(2019, 2, 4), datetime.date(2019, 2, 5),
       datetime.date(2019, 2, 6), datetime.date(2019, 2, 7),
       datetime.date(2019, 2, 8), datetime.date(2019, 2, 12),
       datetime.date(2019, 2, 13), datetime.date(2019, 2, 14),
       datetime.date(2019, 2, 15), datetime.date(2019, 2, 18),
    

### 営業日であるかどうか 

In [12]:
def check_workday_jp(select_date):
    """
    与えられたdatetime.dateが営業日であるかどうかを出力する
    select_date: datetime.date
        入力するdate
    """
    assert isinstance(select_date, datetime.date)
    select_date_array = np.array([select_date])  # データ数が一つのndarray
    # 休日であるかどうか
    is_holiday = np.in1d(select_date_array, option.holidays_date_array).item()  # データ数が一つのため，item
    
    # 土曜日か日曜日であるかどうか
    is_sun_satur = (select_date.weekday()==5) or (select_date.weekday()==6)
    
    is_workday = (not is_holiday) and (not is_sun_satur)
    
    return is_workday

In [13]:
select_date = datetime.date(2019, 1, 4)

check_workday_jp(select_date)

True

### 非営業日を取得

同様に，すでに取得したholidayと照らし合わせる．

In [14]:
def get_not_workdays_jp(start_date, end_date, return_as="date", end_include=False):
    """
    非営業日を取得(土日or祝日)
    
    start_date: datetime.date
        開始時刻のdate
    end_datetime: datetime.date
        終了時刻のdate
    return_as: str, defalt: 'dt'
        返り値の形式
        - 'dt':pd.DatetimeIndex
        - 'date': datetime.date array
    end_include: bool
        最終日も含めて出力するか
    """
    assert isinstance(start_date, datetime.date) and isinstance(end_date, datetime.date)
    # 返り値の形式の指定
    return_as_set = {"dt", "date"}
    if not return_as in return_as_set:
        raise Exception("return_as must be any in {}".format(return_as_set))
    
    # datetime.dateをpd.Timestampに変換(datetime.dateは通常pd.DatetimeIndexと比較できないため)
    start_timestamp = pd.Timestamp(start_date)
    end_timestamp = pd.Timestamp(end_date)
    
    # 期間中のholidayを取得
    holidays_in_span_index = (start_timestamp<=option.holidays_datetimeindex)&(option.holidays_datetimeindex<end_timestamp)  # DatetimeIndexを使うことに注意
    holidays_in_span_array = option.holidays_date_array[holidays_in_span_index]  # ndarrayを使う

    # 期間中のdatetimeのarrayを取得
    if end_include:
        days_datetimeindex = pd.date_range(start=start_date, end=end_date, freq="D")  # 最終日も含める
    else:
        days_datetimeindex = pd.date_range(start=start_date, end=end_date-datetime.timedelta(days=1), freq="D")  # 最終日は含めない
    
    # 休日に含まれないもの，さらに土日に含まれないもののboolインデックスを取得
    holiday_bool_array = np.in1d(days_datetimeindex.date, holidays_in_span_array)  # 休日であるかのブール(pd.DatetimeIndex.isin)でもいい
    sun_or_satur_bool_array = (days_datetimeindex.weekday==5) | (days_datetimeindex.weekday==6)  # 土曜日or日曜日
    
    not_workdays_bool_array = holiday_bool_array | sun_or_satur_bool_array  # 休日あるいは土日
    
    not_workdays_datetimeindex = days_datetimeindex[not_workdays_bool_array].copy()
    if return_as=="dt":
        return not_workdays_datetimeindex
    elif return_as=="date":
        return not_workdays_datetimeindex.date

In [15]:
start_datetime = datetime.date(2019, 1, 1)
end_datetime = datetime.date(2019, 12, 31)

not_workdays = get_not_workdays_jp(start_date, end_date, return_as="dt")

In [16]:
not_workdays

DatetimeIndex(['2019-01-01', '2019-01-05', '2019-01-06', '2019-01-12',
               '2019-01-13', '2019-01-14', '2019-01-19', '2019-01-20',
               '2019-01-26', '2019-01-27',
               ...
               '2019-11-30', '2019-12-01', '2019-12-07', '2019-12-08',
               '2019-12-14', '2019-12-15', '2019-12-21', '2019-12-22',
               '2019-12-28', '2019-12-29'],
              dtype='datetime64[ns]', length=121, freq=None)

## 指定した日数後の営業日を取得

ジェネレーターを使うので速度注意

In [17]:
def get_next_workday_jp(select_date, return_as="date", days=1):
    """
    select_date: datetime.date
        指定する日時
    return_as: str, defalt: 'dt'
        返り値の形式
        - 'dt':pd.Timstamp
        - 'datetime': datetime.datetime array
    """
    assert isinstance(select_date, datetime.date)
    # 返り値の形式の指定
    return_as_set = {"dt", "date"}
    if not return_as in return_as_set:
        raise Exception("return_as must be any in {}".format(return_as_set))
        
    def get_next_workday_jp_gen(select_date):
        add_days=1
        while True:
            next_day = select_date + datetime.timedelta(days=add_days)
            if check_workday_jp(next_day):
                yield next_day
            add_days += 1
    
    next_workday_gen = get_next_workday_jp_gen(select_date)
    for i in range(days):
        next_day = next(next_workday_gen)
    
    if return_as=="date":
        return next_day
    elif return_as=="dt":
        return pd.Timestamp(next_day)

In [18]:
jst_timezone = timezone("Asia/Tokyo")
select_datetime = jst_timezone.localize(datetime.datetime(2020, 1, 1, 0, 0, 0))

next_workday = get_next_workday_jp(select_datetime, days=6, return_as="dt")
next_workday

Timestamp('2020-01-09 00:00:00+0900', tz='Asia/Tokyo')

## データフレームに関しての処理

### データの用意 

awareなdatetimeを持つDataFrame

In [19]:
with open("aware_stock_df.pickle", "rb") as f:
    aware_stock_df = pickle.load(f)

aware_stock_df

,Open_6502,High_6502,Low_6502,Close_6502,Volume_6502
timestamp,,,,,
2020-11-04 00:00:00+09:00,NaN,NaN,NaN,NaN,NaN
2020-11-04 00:01:00+09:00,NaN,NaN,NaN,NaN,NaN
2020-11-04 00:02:00+09:00,NaN,NaN,NaN,NaN,NaN
2020-11-04 00:03:00+09:00,NaN,NaN,NaN,NaN,NaN
2020-11-04 00:04:00+09:00,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
2020-11-30 23:55:00+09:00,NaN,NaN,NaN,NaN,NaN
2020-11-30 23:56:00+09:00,NaN,NaN,NaN,NaN,NaN
2020-11-30 23:57:00+09:00,NaN,NaN,NaN,NaN,NaN


naiveなdatetimeを持つDataFrame

In [20]:
with open("naive_stock_df.pickle", "rb") as f:
    naive_stock_df = pickle.load(f)

naive_stock_df.at_time(datetime.time(9,0))

,Open_6502,High_6502,Low_6502,Close_6502,Volume_6502
timestamp,,,,,
2020-11-04 09:00:00,2669.0,2670.0,2658.0,2664.0,93000.0
2020-11-05 09:00:00,2631.0,2641.0,2630.0,2639.0,55000.0
2020-11-06 09:00:00,2615.0,2644.0,2609.0,2617.0,90700.0
2020-11-09 09:00:00,2711.0,2719.0,2707.0,2710.0,0.0
2020-11-10 09:00:00,2780.0,2782.0,2770.0,2777.0,142700.0
2020-11-11 09:00:00,NaN,NaN,NaN,NaN,NaN
2020-11-12 09:00:00,2788.0,2800.0,2781.0,2789.0,108400.0
2020-11-13 09:00:00,2750.0,2754.0,2741.0,2750.0,71500.0
2020-11-14 09:00:00,NaN,NaN,NaN,NaN,NaN


### 営業日データを取得

高速化のため，既存のholidayと照らし合わせる方法でとる

In [21]:
def extract_workdays_jp_index(dt_index, return_as="index"):
    """
    pd.DatetimeIndexから，営業日のデータのものを抽出
    dt_index: pd.DatetimeIndex
        入力するDatetimeIndex，すでにdatetimeでソートしていることが前提
    return_as: str
        出力データの形式
        - "index": 引数としたdfの対応するインデックスを返す
        - "bool": 引数としたdfに対応するboolインデックスを返す
    """
    # 返り値の形式の指定
    return_as_set = {"index", "bool"}
    if not return_as in return_as_set:
        raise Exception("return_as must be any in {}".format(return_as_set))
        
    # すでにtimestampでソートさてている前提
    start_datetime = dt_index[0].to_pydatetime()
    end_datetime = dt_index[-1].to_pydatetime()
    
    start_datetime, end_datetime = check_jst_datetimes_to_naive(start_datetime, end_datetime)  # 二つのdatetimeのタイムゾーンをチェック・naiveに変更
    
    # 期間内のholidayを取得
    holidays_in_span_index = ((start_datetime-datetime.timedelta(days=1))<option.holidays_datetimeindex)&\
    (option.holidays_datetimeindex<=end_datetime)  # DatetimeIndexを使うことに注意, 当日を含めるため，startから1を引いている．
    holidays_in_span_array = option.holidays_date_array[holidays_in_span_index]  # ndarrayを使う
    
    # 休日に含まれないもの，さらに土日に含まれないもののboolインデックスを取得    
    holiday_bool_array = np.in1d(dt_index.date, holidays_in_span_array)  # 休日
    sun_or_satur_bool_array = (dt_index.weekday==5) | (dt_index.weekday==6)  # 土曜日or日曜日
    
    workdays_bool_array = (~holiday_bool_array)&(~sun_or_satur_bool_array)  # 休日でなく土日でない
    if return_as=="bool":  # boolで返す場合
        return workdays_bool_array
    
    elif return_as=="index":  # indexで返す場合
        workdays_df_indice = dt_index[workdays_bool_array]
        return workdays_df_indice

In [22]:
def extract_workdays_jp(df, return_as="df"):
    """
    データフレームから，営業日のデータのものを抽出．出力データ形式をreturn_asで指定する．
    df: pd.DataFrame(インデックスとしてpd.DatetimeIndex)
        入力データ
    return_as: str
        出力データの形式
        - "df": 抽出した新しいpd.DataFrameを返す
        - "index": 引数としたdfの対応するインデックスを返す
        - "bool": 引数としたdfに対応するboolインデックスを返す
    """
    
    # 返り値の形式の指定
    return_as_set = {"df", "index", "bool"}
    if not return_as in return_as_set:
        raise Exception("return_as must be any in {}".format(return_as_set))
    
    workdays_bool_array = extract_workdays_jp_index(df.index, return_as="bool")
    if return_as=="bool":
        return workdays_bool_array
    
    workdays_df_indice = df.index[workdays_bool_array]
    if return_as=="index":
        return workdays_df_indice

    out_df = df.loc[workdays_df_indice].copy()
    return out_df

In [23]:
extracted_stock_df = extract_workdays_jp(aware_stock_df, return_as="df")
extracted_stock_df.at_time(datetime.time(9,0))

,Open_6502,High_6502,Low_6502,Close_6502,Volume_6502
timestamp,,,,,
2020-11-04 09:00:00+09:00,2669.0,2670.0,2658.0,2664.0,93000.0
2020-11-05 09:00:00+09:00,2631.0,2641.0,2630.0,2639.0,55000.0
2020-11-06 09:00:00+09:00,2615.0,2644.0,2609.0,2617.0,90700.0
2020-11-09 09:00:00+09:00,2711.0,2719.0,2707.0,2710.0,0.0
2020-11-10 09:00:00+09:00,2780.0,2782.0,2770.0,2777.0,142700.0
2020-11-11 09:00:00+09:00,NaN,NaN,NaN,NaN,NaN
2020-11-12 09:00:00+09:00,2788.0,2800.0,2781.0,2789.0,108400.0
2020-11-13 09:00:00+09:00,2750.0,2754.0,2741.0,2750.0,71500.0
2020-11-16 09:00:00+09:00,2714.0,2716.0,2705.0,2710.0,111000.0


### 日中データを取得

おそらく使わない，以下の営業日＋日中内で使う．

In [24]:
def extract_intraday_jp_index(dt_index, return_as="index"):
    """
    pd.DatetimeIndexから，日中(9時から11時半，12時半から15時)のデータのものを抽出．出力データ形式をreturn_asで指定する．
    dt_index: pd.DatetimeIndex
        入力するDatetimeIndex
    return_as: str
        出力データの形式
        - "index": 引数としたdfの対応するインデックスを返す
        - "bool": 引数としたdfに対応するboolインデックスを返す
    """
    
    # 返り値の形式の指定
    return_as_set = {"index", "bool"}
    if not return_as in return_as_set:
        raise Exception("return_as must be any in {}".format(return_as_set))    
  
    bool_array = np.array([False]*len(dt_index))
    
    # 午前をTrueに
    am_indice = dt_index.indexer_between_time(start_time=datetime.time(9,0), end_time=datetime.time(11,30), include_end=False)
    bool_array[am_indice] = True
    
    # 午後をTrueに
    pm_indice = dt_index.indexer_between_time(start_time=datetime.time(12,30), end_time=datetime.time(15,0), include_end=False)
    bool_array[pm_indice] = True
    
    if return_as=="bool":
        return bool_array

    elif return_as=="index":
        intraday_indice = dt_index[bool_array]
        return intraday_indice

In [25]:
def extract_intraday_jp(df, return_as="df"):
    """
    データフレームから，日中(9時から11時半，12時半から15時)のデータのものを抽出．出力データ形式をreturn_asで指定する．
    df: pd.DataFrame(インデックスとしてpd.DatetimeIndex)
        入力データ
    return_as: str
        出力データの形式
        - "df": 抽出した新しいpd.DataFrameを返す
        - "index": 引数としたdfの対応するインデックスを返す
        - "bool": 引数としたdfに対応するboolインデックスを返す
    """
    
    # 返り値の形式の指定
    return_as_set = {"df", "index", "bool"}
    if not return_as in return_as_set:
        raise Exception("return_as must be any in {}".format(return_as_set))    
  
    intraday_bool_array = extract_intraday_jp_index(df.index, return_as="bool")
    if return_as=="bool":
        return intraday_bool_array
    
    intraday_indice = df.index[intraday_bool_array]
    if return_as=="index":
        return intraday_indice
    
    out_df = df.loc[intraday_indice].copy()
    return out_df

以下の例では，朝8時のデータは存在しない

In [26]:
extracted_df = extract_intraday_jp(naive_stock_df, return_as="df")
extracted_df.at_time(datetime.time(8,0))

,Open_6502,High_6502,Low_6502,Close_6502,Volume_6502
timestamp,,,,,


### 営業日＋日中データを取得 

In [27]:
def extract_workdays_intraday_jp_index(dt_index, return_as="index"):
    """
    pd.DatetimeIndexから，営業日+日中(9時から11時半，12時半から15時)のデータのものを抽出．出力データ形式をreturn_asで指定する．
    dt_index: pd.DatetimeIndex
        入力するDatetimeIndex
    return_as: str
        出力データの形式
        - "index": 引数としたdfの対応するインデックスを返す
        - "bool": 引数としたdfに対応するboolインデックスを返す
    """

    # 返り値の形式の指定
    return_as_set = {"index", "bool"}
    if not return_as in return_as_set:
        raise Exception("return_as must be any in {}".format(return_as_set))
        
    workday_bool_array = extract_workdays_jp_index(dt_index, return_as="bool")
    intraday_bool_array = extract_intraday_jp_index(dt_index, return_as="bool")
    
    workday_intraday_bool_array = workday_bool_array & intraday_bool_array
    if return_as=="bool":
        return workday_intraday_bool_array
    elif return_as=="index":
        workday_intraday_indice = dt_index[workday_intraday_bool_array]
        return workday_intraday_indice

In [28]:
def extract_workdays_intraday_jp(df, return_as="df"):
    """
    データフレームから，営業日+日中(9時から11時半，12時半から15時)のデータのものを抽出．出力データ形式をreturn_asで指定する．
    df: pd.DataFrame(インデックスとしてpd.DatetimeIndex)
        入力データ
    return_as: str
        出力データの形式
        - "df": 抽出した新しいpd.DataFrameを返す
        - "index": 引数としたdfの対応するインデックスを返す
        - "bool": 引数としたdfに対応するboolインデックスを返す
    """
    
    # 返り値の形式の指定
    return_as_set = {"df", "index", "bool"}
    if not return_as in return_as_set:
        raise Exception("return_as must be any in {}".format(return_as_set))    
       
    workday_intraday_bool_array = extract_workdays_intraday_jp_index(df.index, return_as="bool")
    
    if return_as=="bool":
        return workday_intraday_bool_array
    
    workday_intraday_indice = df.index[workday_intraday_bool_array]
    
    if return_as=="index":
        return workday_intraday_indice
    
    out_df = df.loc[workday_intraday_indice].copy()
    return out_df

In [29]:
extracted_df = extract_workdays_intraday_jp(aware_stock_df, return_as="df")
extracted_df.at_time(datetime.time(9,0))

,Open_6502,High_6502,Low_6502,Close_6502,Volume_6502
timestamp,,,,,
2020-11-04 09:00:00+09:00,2669.0,2670.0,2658.0,2664.0,93000.0
2020-11-05 09:00:00+09:00,2631.0,2641.0,2630.0,2639.0,55000.0
2020-11-06 09:00:00+09:00,2615.0,2644.0,2609.0,2617.0,90700.0
2020-11-09 09:00:00+09:00,2711.0,2719.0,2707.0,2710.0,0.0
2020-11-10 09:00:00+09:00,2780.0,2782.0,2770.0,2777.0,142700.0
2020-11-11 09:00:00+09:00,NaN,NaN,NaN,NaN,NaN
2020-11-12 09:00:00+09:00,2788.0,2800.0,2781.0,2789.0,108400.0
2020-11-13 09:00:00+09:00,2750.0,2754.0,2741.0,2750.0,71500.0
2020-11-16 09:00:00+09:00,2714.0,2716.0,2705.0,2710.0,111000.0


In [30]:
extracted_df.at_time(datetime.time(8,0))

,Open_6502,High_6502,Low_6502,Close_6502,Volume_6502
timestamp,,,,,
